# Dependencies

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

# Targets:

1. Fix data types of the dataset [dtypes] --> [done]
2. Fix column names to not have spaces and have short names. [Done]
    
    2.1 Also, prepare data dictionary for new column names. [Done]
3. Create better features [by combining some columns]
    * Try to get the CO2 emissions from the consumption
4. Check co-relation and highlight highly correlated variables
5. Remove correlated variables
6. Get Count, Average, min and max for each month by each year of all numerical fields.
7. ***[Advanced step]*** Based on all above steps compute relative change:
    7.1 Monthly
    7.2 Yearly
8. Data Standardization
    * Get the features to same scale
9. Filling the Folumula based Columns
10. Supply - Demand theory
    * Matching the theory
    * Optimize to get a best value for supply [Probably least square, root finding]
    * Visualize these results for states
    * Convert the output to percentage to generalize
    
    * $\begin{align}\\ E_w = 1.93 * V^3 * R^2 * C_p \\\\\end{align}$  (in Watts)
    * $\begin{align}\\ E_s = 1367 * K_t * R^2 * C_p \\\\\end{align}$  (in Watts)
11. Document assumptions:
    * $\begin{align}\\ Wind: C_p ~element ~of ~[0.25, 0.45] @ max ~ -> 0.59 \\\\\end{align}$
        * so, we are taking the average value to be 0.35
    
    * $\begin{align}\\ Solar: C_p ~element ~of ~[0.24, 0.34] @ max ~ -> 0.46 \\\\\end{align}$
        * so, we are taking the average value to be 0.29

# Resources

This is a good introductory python repository for Exploratory Data Analysis.

<a href="https://github.com/cmawer/pycon-2017-eda-tutorial/tree/master/notebooks"> EDA Tutorial</a>

# Reading Data

In [2]:
# Reading the data
df = pd.read_csv('data.csv')

# creating a Period column for 
df["period"] = (df["Year"].map(str) + '-' + df["Month"].map(str))

# Renaming the columns by replacing spaces and adding underscores
'''
Reason: for better accessibility of columns
'''
col = df.columns
col = [x.replace(" ", "_") for x in col]
col = [x.replace("(", "") for x in col]
col = [x.replace(")", "") for x in col]
col = [x.replace(",", "") for x in col]
col = [x.replace("^", "") for x in col]
col = [x.replace("*", "") for x in col]
col = [x.replace(".", "") for x in col]
col = [x.replace("=", "_") for x in col]
df.columns = col

# changed the option of head to display all columns
ncol = df.shape[1]
pd.set_option('display.max_columns', ncol*2)

# Let See our data
df.head(2)

,Year,Month,Net_generation_other_renewables_total__thousand_megawatthours,Average_retail_price_of_electricity_cents_per_KWH,Retail_sales_of_electricity_million_kilowatthours,Revenue_from_retail_sales_of_electricity_totalmillion_dollars,Revenue_from_retail_sales_of_electricity_residential_million_dollars,Total_consumption_coal_thousand_tons,Total_consumption_petroleum_liquid_thousand_barrels,Total_consumption__natural_gas_thousand_Mcf,unqid,kt_avg,T2M_avg,WS10M_avg,State,kt_max,T2M_max,WS10M_max,kt_min,T2M_min,WS10M_min,for_wind__Power___193__V3__R2__Cp__in_watts,for_solar__1367__kt__r__2__efficiency,total_renewable_policy_count,Wind_policy_count,Solar_photovoltaic_policy_count,period
0,2017,7,303.596600,12.53,3429.51828,878.76993,429.58206,1858.693000,3.938000,42893.952000,2017_7_Alabama,0.558278,27.427392,1.380161,Alabama,0.71,30.42,3.2,0.08,24.62,0.43,"Cp is max power coefficient , varies from 0.25...",r is length of solar array,37,5,7,2017-7
1,2017,6,279.922490,12.79,2727.88024,766.67053,348.87341,1489.063000,2.741000,33357.656000,2017_6_Alabama,0.470984,24.293833,1.712028,Alabama,0.71,27.62,4.2,0.06,19.44,0.79,r is radius of swept area in meters,"efficiency , varies from 14 to 24 % , max of ...",37,5,7,2017-6


In [3]:
# Percentage of how many nulls we have?
df.isnull().sum()/df.shape[0]

Year                                                                    0.000000
Month                                                                   0.000000
Net_generation_other_renewables_total__thousand_megawatthours           0.000000
Average_retail_price_of_electricity_cents_per_KWH                       0.000000
Retail_sales_of_electricity_million_kilowatthours                       0.000000
Revenue_from_retail_sales_of_electricity_totalmillion_dollars           0.000000
Revenue_from_retail_sales_of_electricity_residential_million_dollars    0.000000
Total_consumption_coal_thousand_tons                                    0.000000
Total_consumption_petroleum_liquid_thousand_barrels                     0.000000
Total_consumption__natural_gas_thousand_Mcf                             0.000000
unqid                                                                   0.000000
kt_avg                                                                  0.000000
T2M_avg                     

In [4]:
# Converting object type to Float!
def to_float(df, col):
    return pd.to_numeric(df[col].str.replace(' ',''), errors='force')

column_name = 'Net_generation_other_renewables_total__thousand_megawatthours'
df[column_name] = to_float(df = df, col= column_name)

column_name = 'Total_consumption_coal_thousand_tons'
df[column_name] = to_float(df = df, col= column_name)

column_name = 'Total_consumption_petroleum_liquid_thousand_barrels'
df[column_name] = to_float(df = df, col= column_name)

column_name = 'Total_consumption__natural_gas_thousand_Mcf'
df[column_name] = to_float(df = df, col= column_name)

In [5]:
# Check only if having issues with datatypes or, want to learn about data types of columns
df.dtypes

Year                                                                      int64
Month                                                                     int64
Net_generation_other_renewables_total__thousand_megawatthours           float64
Average_retail_price_of_electricity_cents_per_KWH                       float64
Retail_sales_of_electricity_million_kilowatthours                       float64
Revenue_from_retail_sales_of_electricity_totalmillion_dollars           float64
Revenue_from_retail_sales_of_electricity_residential_million_dollars    float64
Total_consumption_coal_thousand_tons                                    float64
Total_consumption_petroleum_liquid_thousand_barrels                     float64
Total_consumption__natural_gas_thousand_Mcf                             float64
unqid                                                                    object
kt_avg                                                                  float64
T2M_avg                                 

In [6]:
df['Net_generation_other_renewables_total__thousand_megawatthours'] = df['Net_generation_other_renewables_total__thousand_megawatthours']*1000
df['Average_retail_price_of_electricity_cents_per_KWH'] = (df['Average_retail_price_of_electricity_cents_per_KWH']*0.100) #*0.001)*0.01
df['Retail_sales_of_electricity_million_kilowatthours'] = (df['Retail_sales_of_electricity_million_kilowatthours']*1000000)*0.001
df['Revenue_from_retail_sales_of_electricity_totalmillion_dollars'] = df['Revenue_from_retail_sales_of_electricity_totalmillion_dollars']*1000000 
df['Revenue_from_retail_sales_of_electricity_residential_million_dollars'] = df['Revenue_from_retail_sales_of_electricity_residential_million_dollars']*1000000                                                                    
df['Total_consumption_coal_thousand_tons'] = df['Total_consumption_coal_thousand_tons']*1000*8.141
df['Total_consumption_petroleum_liquid_thousand_barrels'] = df['Total_consumption_petroleum_liquid_thousand_barrels']*1000*0.58844074839725
df['Total_consumption__natural_gas_thousand_Mcf'] = df['Total_consumption__natural_gas_thousand_Mcf']*1000*0.00029307107017222

In [7]:
# sorting the dataframe by year + month = Period
df = df.sort_values(['period'], ascending=[1])

# curating for_wind__Power_=_1.93__V3__R2__Cp__in_watts:
r = 35.3568
c_w_p = 0.35
df['for_wind__Power___193__V3__R2__Cp__in_watts'] = 1.93 * df.WS10M_min**3 * r**2 * c_w_p


# curating ffor_solar__1367__kt__r__2__efficiency:
r = 1
c_s_p = 0.29
df['for_solar__1367__kt__r__2__efficiency'] = 1367 * df.kt_min * r**2 * c_s_p

df.head()

,Year,Month,Net_generation_other_renewables_total__thousand_megawatthours,Average_retail_price_of_electricity_cents_per_KWH,Retail_sales_of_electricity_million_kilowatthours,Revenue_from_retail_sales_of_electricity_totalmillion_dollars,Revenue_from_retail_sales_of_electricity_residential_million_dollars,Total_consumption_coal_thousand_tons,Total_consumption_petroleum_liquid_thousand_barrels,Total_consumption__natural_gas_thousand_Mcf,unqid,kt_avg,T2M_avg,WS10M_avg,State,kt_max,T2M_max,WS10M_max,kt_min,T2M_min,WS10M_min,for_wind__Power___193__V3__R2__Cp__in_watts,for_solar__1367__kt__r__2__efficiency,total_renewable_policy_count,Wind_policy_count,Solar_photovoltaic_policy_count,period
9083,2001,1,25784.00,0.601,250605.91,48128400.0,15071360.0,1.886738e+07,2788.620707,66.972894,2001_1_Wyoming,0.580634,7.877419,3.428387,Wyoming,0.79,20.49,7.33,0.21,0.05,0.78,400.732961,83.2503,38,3,2,2001-1
2128,2001,1,59135.88,0.753,4276214.87,727441860.0,321988990.0,3.496479e+07,19181.991516,320.475853,2001_1_Illinois,0.496512,4.975000,3.038508,Illinois,0.79,16.24,5.95,0.07,0.00,0.57,156.385263,27.7501,93,13,16,2001-1
4705,2001,1,5112.00,0.644,441087.94,62825520.0,28420550.0,8.031324e+06,285.393763,0.518736,2001_1_Montana,0.513096,6.872903,4.230335,Montana,0.76,18.65,10.94,0.17,0.01,0.69,277.407711,67.3931,48,13,14,2001-1
1193,2001,1,9318.00,0.671,1421031.41,214823030.0,95360420.0,1.446262e+07,23172.796672,1765.353449,2001_1_Colorado,0.586164,6.415760,3.230219,Colorado,0.81,20.49,9.25,0.06,0.01,0.88,575.465475,23.7858,134,18,47,2001-1
4506,2001,1,865.62,0.587,3233670.91,365038460.0,189790380.0,2.938603e+07,43959.466109,142.258456,2001_1_Missouri,0.482445,3.415280,2.956570,Missouri,0.80,16.47,6.36,0.03,0.02,0.96,747.110700,11.8929,79,10,16,2001-1


In [8]:
col

['Year',
 'Month',
 'Net_generation_other_renewables_total__thousand_megawatthours',
 'Average_retail_price_of_electricity_cents_per_KWH',
 'Retail_sales_of_electricity_million_kilowatthours',
 'Revenue_from_retail_sales_of_electricity_totalmillion_dollars',
 'Revenue_from_retail_sales_of_electricity_residential_million_dollars',
 'Total_consumption_coal_thousand_tons',
 'Total_consumption_petroleum_liquid_thousand_barrels',
 'Total_consumption__natural_gas_thousand_Mcf',
 'unqid',
 'kt_avg',
 'T2M_avg',
 'WS10M_avg',
 'State',
 'kt_max',
 'T2M_max',
 'WS10M_max',
 'kt_min',
 'T2M_min',
 'WS10M_min',
 'for_wind__Power___193__V3__R2__Cp__in_watts',
 'for_solar__1367__kt__r__2__efficiency',
 'total_renewable_policy_count',
 'Wind_policy_count',
 'Solar_photovoltaic_policy_count',
 'period']

In [9]:
# Add CO2 emissions for Coal, Petrol, and Natural Gas
# didn't write this correctly but the calculation are right! lol sorry
df['Kg_CO2_emitted_from_Coal'] = (((df['Total_consumption_coal_thousand_tons']*3409510.6405145)/1000000)*215)*0.45359237
df['Kg_CO2_emitted_from_Petrol'] = (((df['Total_consumption_petroleum_liquid_thousand_barrels']*3409510.6405145)/1000000)*157.2)*0.45359237
df['Kg_CO2_emitted_from_NaturalGas'] = (((df['Total_consumption__natural_gas_thousand_Mcf']*3409510.6405145)/1000000)*117)*0.45359237

In [10]:
df.head()
df.to_csv('data_690.csv')

In [11]:
df.head(10)
import numpy as np
np.mean(df.WS10M_avg)

3.384079372092801

In [12]:
df.sample(5)
# df.dtypes  # need to work on this

,Year,Month,Net_generation_other_renewables_total__thousand_megawatthours,Average_retail_price_of_electricity_cents_per_KWH,Retail_sales_of_electricity_million_kilowatthours,Revenue_from_retail_sales_of_electricity_totalmillion_dollars,Revenue_from_retail_sales_of_electricity_residential_million_dollars,Total_consumption_coal_thousand_tons,Total_consumption_petroleum_liquid_thousand_barrels,Total_consumption__natural_gas_thousand_Mcf,unqid,kt_avg,T2M_avg,WS10M_avg,State,kt_max,T2M_max,WS10M_max,kt_min,T2M_min,WS10M_min,for_wind__Power___193__V3__R2__Cp__in_watts,for_solar__1367__kt__r__2__efficiency,total_renewable_policy_count,Wind_policy_count,Solar_photovoltaic_policy_count,period,Kg_CO2_emitted_from_Coal,Kg_CO2_emitted_from_Petrol,Kg_CO2_emitted_from_NaturalGas
1702,2003,5,369172.40,0.856,8652800.14,1.332868e+09,740557600.0,2.108827e+07,3.054858e+06,14917.514122,2003_5_Florida,0.585558,24.770670,3.020025,Florida,0.75,26.93,7.31,0.02,21.09,0.69,277.407711,7.9286,104,10,26,2003-5,7.011923e+09,7.426793e+08,2.699231e+06
5530,2015,3,198211.83,1.205,496537.54,1.662884e+08,59817400.0,6.932135e+06,5.925010e+03,1846.523292,2015_3_NewMexico,0.630747,9.216978,3.779258,NewMexico,0.81,20.16,9.23,0.09,0.01,1.37,2171.365627,35.6787,63,16,25,2015-3,2.304959e+09,1.440454e+06,3.341169e+05
3131,2016,2,426227.23,1.524,407542.91,1.211363e+08,62095980.0,4.941587e+04,5.568543e+04,555.803716,2016_2_Maine,0.442931,7.477385,3.679195,Maine,0.76,24.53,11.51,0.03,0.03,1.47,2682.398277,11.8929,31,10,11,2016-2,1.643095e+07,1.353792e+07,1.005692e+05
3341,2015,3,99655.06,1.307,2550075.85,6.478828e+08,333307090.0,4.037268e+06,1.619095e+04,395.849629,2015_3_Maryland,0.437258,4.615161,4.743065,Maryland,0.75,12.60,8.14,0.04,0.01,1.78,4762.459156,15.8572,88,17,32,2015-3,1.342406e+09,3.936249e+06,7.162653e+04
2643,2007,11,103441.00,0.767,923716.95,1.934838e+08,70859790.0,1.580498e+07,6.382228e+03,194.618240,2007_11_Kansas,0.575059,7.227492,3.602333,Kansas,0.72,17.51,7.50,0.14,0.03,0.80,432.355729,55.5002,26,5,8,2007-11,5.255211e+09,1.551610e+06,3.521496e+04


# Machine Intelligence!